# Fonctions et variables utilisées dans anomalies

In [2]:
# Imortation des dataframes de base
%run dataframes.ipynb

In [ ]:
### GENERATION DES ECARTS ###

# Construction d'un df contenant les différents types écarts de chaque thème
def ecarts_df_f(count_df):
    ecarts = []
    for theme in themes:
        vecteur = count_df[theme].values
        moyenne = np.mean(vecteur)
        ecartAbs = max(vecteur-moyenne)
        ecartRel = ecartAbs/moyenne
        ecartSqrt = ecartAbs/np.sqrt(moyenne)
        ecarts.append({
            "theme":theme,
            "vecteur":vecteur,
            "moyenne": moyenne,
            "ecartAbs" : ecartAbs,
            "ecartRel" : ecartRel,
            "ecartSqrt" : ecartSqrt
        })
    return pd.DataFrame(ecarts).set_index('theme')

In [ ]:
ecarts = ecarts_df_f(count_df)

In [ ]:
### AFFICHAGES ###

# Affichage des k thèmes avec leur.s période.s correspondante.s
def show_count_df(themes_to_show, k = 4, c = 2, figsize = (20,12)):
    fig, axes = plt.subplots(ncols = c, nrows = k//c+k%c,figsize = figsize)
    for i in range(k):
        theme = themes_to_show[i]
        axe = axes[i//c,i%c]
        try:
            count_df.plot(y=theme,ax=axe)
        except Exception as e:
            print('Could\'nt print <',theme, '> because :', e)

In [ ]:
### DETECTIONS DES PICS ###

# Détection du plus gros pic
def max_pic_date(theme):
    return count_df[theme].idxmax(axis=0)

# Dates des pics repérés, selon la méthode utilisée
def pics_dates(theme, method = 'max'):
    if method == 'max':
        return [max_pic_date(theme)]

In [ ]:
### DETECTIONS DES PERIODES CORRESPONDANTES AUX PICS ###

# Détection de la période correspondante à un pic
def pic_period_above_mean(theme,pic_date):
    pre_pic, post_pic = pic_date, pic_date
    m = ecarts['moyenne'][theme]
    dates_inf = count_df[count_df[theme]<m].index
    for i_date in range(len(dates_inf)):
        date = dates_inf[i_date]
        if date > pic_date:
            post_pic = date
            pre_pic = dates_inf[i_date-1]
            break
    print('Periode:',pre_pic,post_pic)
    return (pre_pic,post_pic)

# Détection de la période correspondante à un pic, selon la méthode utilisée
def pic_period(theme, pic_date, method = 'above_mean'):
    if method == 'above_mean':
        return pic_period_above_mean(theme,pic_date)

In [1]:
### UTILES A L'ANALYSE ###

# Fonction de repérage de thèmes 
def reperage_themes(m_meilleurs_themes = 'ecartSqrt', m_detect_pic = 'max', m_period_pic = 'above_mean', nb_to_show = 4, aleatoire  = False, show_figure = True, c = 2):
    # On prend nb_to_show thèmes aléatoires :
    if aleatoire:
        themes_to_show = [themes[i] for i in np.random.randint(len(themes),size = k)]
    # Ou bien les nb_to_show thèmes avec les meilleurs pics :
    else:
        themes_to_show = ecarts.sort_values(ascending = False,by=m_meilleurs_themes).head(nb_to_show).index
        
    # Affichage des figures obtenues:
    if show_figure:
        fig, axes = plt.subplots(ncols = c, nrows = nb_to_show//c+nb_to_show%c,figsize = (30,20))
        for i in range(nb_to_show):
            theme = themes_to_show[i]
            axe = axes[i//c,i%c]
            count_df.plot(y=theme,ax=axe)

            for date in pics_dates(theme,m_detect_pic):
                inf,sup = pic_period(theme,date,m_period_pic)
                try:
                    count_df[(count_df.index>=inf) & (count_df.index<=sup)].plot(y=theme,ax=axe)
                except Exception as e:
                    print('Could\'nt print <',theme, '> because :', e)
                    
    return ecarts.sort_values(ascending = False,by=m_meilleurs_themes).head(nb_to_show)[[m_meilleurs_themes,'moyenne']]

# Renvoie le df de questions d'un theme asssociés à une période 
def df_periode(theme,periode,plot = True):
    mois = pd.Timedelta('30 days')
    debut, fin = periode
    if plot:
        ax = count_df[(count_df.index>=debut - mois) & (count_df.index<=fin + mois)].plot(y=theme,figsize = (12,5))
        count_df[(count_df.index>=debut) & (count_df.index<=fin)].plot(y=theme,ax = ax)
    return questions_df[(questions_df.date_question>=debut) & (questions_df.date_question<=fin) & (questions_df.rubrique == theme)].sort_values('date_question').set_index('date_question')

# Les groupes les plus présents sur un df de questions
def ordre_groupes(data):
    return data.groupby('groupe_auteur').count().sort_values(by='question',ascending = False)[['question','date_reponse']]

# Les ministres les plus adressés sur un df de questions
def ordre_ministres(data):
    return data.groupby('ministere_adresse').count().sort_values(by='question',ascending = False)[['question','date_reponse']]